<h1> Daily DattoRMM Monitor Metrics </h1>

In [ ]:
#data conditioning
import pandas as pd
import re
import datetime as dt


# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# dictionary location
dictionary_dir = f'{git_folder}/dictionaries'

In [ ]:
from ts_api_connections import datto_rmm_api

In [ ]:
datto = datto_rmm_api.DattoRMM()

In [ ]:
df_alerts = datto.create_monitors_dataframe(days=2)

In [ ]:
df_alerts['timeOccured'] = df_alerts['timestamp'].apply(lambda x: x.strftime("%H:%M"))

# Create Visual Documents and Save Docs

In [ ]:
df_ticket_alerts

In [ ]:
df_alerts[df_alerts["deviceName"] == "AWS-PROD-MFA1"]

In [ ]:
df_ticket_alerts = df_alerts[df_alerts['sendsEmails'] == True]

In [ ]:
df_ticket_alerts[df_ticket_alerts['diagnostics'] == ""].value_counts('deviceName')

In [ ]:
df_ticket_alerts[df_ticket_alerts['alertUid'] == "[REDACTED_UUID]"]

In [ ]:
df_resource_alerts = df_alerts[(df_alerts["alertClass"] == "eventlog_ctx") | (df_alerts["alertClass"] == "perf_resource_usage_ctx") | (df_alerts["alertClass"] == "online_offline_status_ctx") ]

In [ ]:
df_resource_alerts["alertClass"].replace({"eventlog_ctx":"Memory High", "perf_resource_usage_ctx":"CPU High", "online_offline_status_ctx": "Device Offline"},inplace=True)

In [ ]:
df_resource_alerts.timeOccured.value_counts().reset_index().to_csv("D:/exports/time_occured.csv")

In [ ]:
df_offline_alerts = df_alerts[(df_alerts["alertClass"] == "online_offline_status_ctx") ]
df_offline_alerts.to_csv("D:/exports/offline_alerts_2_days.csv")

In [ ]:
df_ticket_alerts = df_alerts[df_alerts['sendsEmails'] == True]

In [ ]:

fig = plt.figure(figsize=(60,40))
palette = sns.color_palette("rainbow")
sns.countplot(data=df_ticket_alerts[df_ticket_alerts['monitorSource'] == "Offline Alert"].sort_values('timeOccured'),x = 'timeOccured',color='lightblue')
plt.savefig(".png")

In [ ]:
df_offline_alerts = df_ticket_alerts[df_ticket_alerts['monitorSource'] == "Offline Alert"]
df_offline_alerts[df_offline_alerts['timeOccured'].isin(['06:10','17:06','17:40','06:09','17:07','21:44'])][['siteName','deviceName','timeOccured']]

In [ ]:
df_resource_alerts[df_resource_alerts['timestamp'] >= "2022-12-03 00:00:00"].value_counts()

In [ ]:
df_offline_alerts[df_offline_alerts['timestamp'] >= "2022-12-03 00:00:00"].value_counts('deviceName')

In [ ]:
start_date = df_offline_alerts['timestamp'] >= "2022-12-02 00:00:00"
end_date = df_offline_alerts["timestamp"] >= "2022-12-05 00:00:00"

In [ ]:
df_offline_alerts[start_date & end_date].value_counts('deviceName')

In [ ]:
start_date = df_offline_alerts['timestamp'] >= "2022-11-29 00:00:00"
end_date = df_offline_alerts["timestamp"] >= "2022-12-2 00:00:00"

In [ ]:
df_offline_alerts[start_date & end_date].value_counts('deviceName')

In [ ]:
df_offline_alerts[df_offline_alerts['timestamp'] >= "2022-12-01 00:00:00"].value_counts('deviceName')

In [ ]:
fig = plt.figure(figsize=(60,40))
sns.countplot(data=df_ticket_alerts.sort_values('dayOccurred'),x = 'dayOccurred', hue='monitorSource')
fig.savefig(".png")

In [ ]:
fig = plt.figure(figsize=(60,40))
sns.countplot(data=df_alerts.sort_values('dayOccurred'),x = 'dayOccurred', hue='monitorSource')
fig.savefig(".png")

# Save Report to CSV

## All Alerts

### Open Alerts

In [ ]:
df_alerts[df_alerts['resolved'] == False].to_csv(export_folder + 'datto_rmm_open_alerts_' + str(current_time) + '.csv',index=False)

### All Alerts - Last 7 Days

In [ ]:
df_alerts.to_csv(export_folder + 'datto_rmm_7days_alerts_' + str(current_time) + '.csv',index=False)

## Alert Breakdown by Unique Category

In [ ]:
df_alerts['alertStatus'].value_counts()

### Alerts by Client Summary

In [ ]:
df_alerts['siteName'].value_counts()

In [ ]:
df_alerts.shape

## Alert Breakdown by Time to Resolve

<h2> Offline Alert Resolution Time </h2>

In [ ]:
df_offline = df_alerts[df_alerts['monitorSource'] == 'Offline Alert']

In [ ]:
df_offline['timeToResolve'].describe()

In [ ]:
df_offline

In [ ]:
list(df_offline.value_counts('deviceName').head(10).reset_index().drop(0,axis=1).deviceName)
df_offline_mask = df_offline['deviceName'].isin(list(df_offline.value_counts('deviceName').head(10).reset_index().drop(0,axis=1)))

In [ ]:
df_offline.value_counts('deviceName').head(10).reset_index().deviceName

In [ ]:
df_offline

In [ ]:
df_offline.value_counts('deviceName',ascending=False).head(30).reset_index().plot(kind='bar')

In [ ]:
fig = plt.figure(figsize=(60,40))

sns.barplot(data=df_offline.value_counts('deviceName',ascending=False).head(30).reset_index(), x='deviceName')
fig.savefig(".png")

In [ ]:
fig = plt.figure(figsize=(60,40))
sns.countplot(data=df_offline.sort_values('siteName'), x='siteName')
fig.savefig(".png")

<h2> Space Alert Resolution Time </h2>

In [ ]:
df_offline[['siteName','deviceName']].groupby('siteName').value_counts().to_csv('d:/exports/offline_alerts_group_by_sitename')

In [ ]:
df_devices = datto.create_devices_dataframe()

In [ ]:
df_devices[df_devices.suspended == 1].to_csv("d:/exports/suspended_devices.cvs")

In [ ]:
df_alerts

## Memory Alerts

In [ ]:
df_alerts[df_alerts['alertUid'] == "[REDACTED_UUID]"]

In [ ]:
df_memory_alerts = df_alerts[df_alerts[""]]